In [6]:
import numpy as np
import random, sys, time, re
import math
import copy
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, \
ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
import xlsxwriter
import pandas as pd

class Test:

    def __init__(self,problem,alpha,beta):
        self.id=id
        self.data=problem
        self.a=alpha
        self.b=beta   
    
    def nbrUn(self,solution):
        return len([i for i, n in enumerate(solution) if n == 1])
    
    def fitness_eq(self, X):
        return (1-self.data.evaluate(X))*self.a+(self.nbrUn(X)/self.data.nb_attribs)*self.b
# 种群初始化函数 '''
    def initial(self,pop,dim):
        X = np.zeros([pop,dim])
        for i in range(0,pop):
            for j in range(0,dim):
                if random.random() < 0.5:
                    X[i][j] = 0
                else:
                    X[i][j] = 1
        return X


            
#边界检查函数'''
    def BorderCheck(self,X,ub,lb,pop,dim):
        for i in range(pop):
            for j in range(dim):
                if X[i,j]>ub[j]:
                    X[i,j] = ub[j]
                elif X[i,j]<lb[j]:
                    X[i,j] = lb[j]
        return X
     
#秃鹰优化算法'''
    def BES(self,pop,lb,ub,MaxIter):
        dim=self.data.nb_attribs
        X = self.initial(pop,dim) #初始化种群
        fitness = np.array(np.zeros(pop))
        fitnessNew = np.array(np.zeros(pop))
        DeltaX= np.zeros([pop,dim])
        DeltaX1= np.zeros([pop,dim])
        for i in range(0,pop):
            fitness[i]=self.fitness_eq(X[i,:])
        minIndex = np.argmin(fitness)
        Xnew = copy.deepcopy(X)
        GbestScore = copy.deepcopy(fitness[minIndex])
        GbestPositon = np.array(np.zeros(dim, int))
        GbestPositon = copy.deepcopy(X[minIndex,:])
        food_x = np.array(np.zeros(MaxIter))
        for t in range(0,MaxIter):
            #print("第"+str(t)+"次迭代")
            lm = 2
        #1.选择搜索空间
            Mean = np.mean(X)

            for i in range(pop):
                for j in range(dim):
                    DeltaX[i,j]=lm*np.random.random()*(Mean-X[i,j])
                    Xnew[i,j]=GbestPositon[j] + DeltaX[i,j]
                    T = abs(1-2/ (1 +  np.exp(-DeltaX[i,j])))
                    if random.random() < T:
                        Xnew[i,j] =not GbestPositon[j]
                    else:
                        Xnew[i,j]=GbestPositon[j]
                #fitness[i]=self.fitness_eq(X[i,:])
            for i in range(pop):
                fitnessNew[i] = self.fitness_eq(Xnew[i,:])      

            #计算适应度值
            for i in range(pop):
                if fitnessNew[i]<fitness[i]:
                    fitness[i] = copy.deepcopy(fitnessNew[i])
                    X[i,:] = copy.deepcopy(Xnew[i,:])
            minIndex = np.argmin(fitness)
            if fitness[minIndex]<GbestScore:
                GbestScore = copy.deepcopy(fitness[minIndex])
                GbestPositon = copy.deepcopy(X[minIndex,:])
        
        #2.搜索猎物
            Mean = np.mean(X)
            a = 10
            R = 1.5
            c1=1.25
            c2=1.5
            w=0.7298
            for i in range(pop-1):
                for j in range(dim):
                    th = a*np.pi*np.random.random([pop,1])
                    r = th + R*np.random.random([pop,1])
                    xR = r*np.sin(th)
                    yR = r*np.cos(th)
                    x=xR/np.max(np.abs(xR))
                    y=yR/np.max(np.abs(yR))
                    DeltaX1[i,j]=y[i]*(X[i,j]-X[i+1,j])+x[i]*(X[i,j]-Mean)
                    DeltaX[i,j]=w*X[i,j]+np.random.random()*c1*(DeltaX1[i,j]-X[i,j])+np.random.random()*c2*(GbestPositon[j]-X[i,j])
                    T = abs(1-2/ (1 +  np.exp(-DeltaX[i,j])))
                    if random.random() < T:
                        Xnew[i,j] =not X[i,j]
                    else:
                        Xnew[i,j]=X[i,j]  

            for i in range(pop):
                if fitnessNew[i]<fitness[i]:
                    fitness[i] = copy.deepcopy(fitnessNew[i])
                    X[i,:] = copy.deepcopy(Xnew[i,:])
            minIndex = np.argmin(fitness)
            if fitness[minIndex]<GbestScore:
                GbestScore = copy.deepcopy(fitness[minIndex])
                GbestPositon = copy.deepcopy(X[minIndex,:])
        
        #3.俯冲捕获猎物
            Mean = np.mean(X)
            a = 10
            R = 1.5
            for i in range(pop):
                for j in range(dim):
                    th = a*np.pi*np.random.random([pop,1])
                    r = th
                    xR = r*np.sinh(th)
                    yR = r*np.cosh(th)
                    x=xR/np.max(np.abs(xR))
                    y=yR/np.max(np.abs(yR))
                    DeltaX1[i,j]=y[i]*(X[i,j]-2*Mean)+x[i]*(X[i,j]-2*GbestPositon[j])
                    DeltaX[i,j]=w*X[i,j]+np.random.random()*c1*(DeltaX1[i,j]-X[i,j])+np.random.random()*c2*(GbestPositon[j]-X[i,j])
                   
                    T = abs(1-2/ (1 +  np.exp(-DeltaX[i,j])))
                    
                    if random.random() < T:
                        Xnew[i,j] =not X[i,j]
                    else:
                        Xnew[i,j]=X[i,j]
                        
            for i in range(pop):
                if fitnessNew[i]<fitness[i]:
                    fitness[i] = copy.deepcopy(fitnessNew[i])
                    X[i,:] = copy.deepcopy(Xnew[i,:])
            minIndex = np.argmin(fitness)
            if fitness[minIndex]<GbestScore:
                GbestScore = copy.deepcopy(fitness[minIndex])
                GbestPositon = copy.deepcopy(X[minIndex,:])
      
           # Curve[t] = GbestScore
            food_x[t]=GbestScore
        return self.data.evaluate(GbestPositon),self.nbrUn(GbestPositon),GbestScore,food_x
        #return GbestScore,GbestPositon,food_x
class FsProblem :
    def __init__(self,data):
        self.data=data
        self.nb_attribs= len(self.data.columns)-1       
        self.outPuts=self.data.iloc[:,self.nb_attribs]     
        self.classifier = KNeighborsClassifier(n_neighbors=5)
    
    def evaluate(self,solution):
        list=[i for i, n in enumerate(solution) if n == 1]
        if (len(list)== 0):
            return 0
        df = self.data.iloc[:,list]        
        array=df.values
        nbrAttributs =len(array[0])
        X = array[:,0:nbrAttributs]
        Y = self.outPuts
        cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
        results = cross_val_score(self.classifier, X, Y, cv=cv,scoring='accuracy')

        return results.mean()
    

class FSData:

    def __init__(self,location,nbr_exec,dragonfly_num,max_iter):
        
        self.location = location
        self.nb_exec = nbr_exec
        self.dataset_name = re.search('[A-Za-z\-]*.csv',self.location)[0].split('.')[0]
        self.df = pd.read_csv(self.location,header=None)
        
        self.fsd = FsProblem(self.df)
        self.dragonfly_num=dragonfly_num
        self.max_iter=max_iter
        
        self.classifier_name = str(type(self.fsd.classifier)).strip('< > \' class ').split('.')[3]
        path = '../sheets/'+ self.dataset_name
        self.instance_name = str("HBBP"+time.strftime("%m-%d-%H-%M_", time.localtime()))
        log_filename = str('../logs/'+self.dataset_name + "/" + self.instance_name)   
        sheet_filename = str(path + '/' +self.instance_name )
        
        log_file = open(log_filename + '.txt','w+')
        sys.stdout = log_file
        
        print("[START] Dataset" + self.dataset_name + "description \n")
        print("Shape : " + str(self.df.shape) + "\n")
        print(self.df.describe())
        print("\n[END] Dataset" + self.dataset_name + "description\n")
        print("[START] Ressources specifications\n")
        print("[END] Ressources specifications\n")

        
        self.workbook = xlsxwriter.Workbook(sheet_filename + '.xlsx')
        
        self.worksheet = self.workbook.add_worksheet(self.classifier_name)
        self.worksheet2 = self.workbook.add_worksheet('fitness')
        self.worksheet.write(0,0,'Iteration')
        self.worksheet.write(0,1,'Accuracy')
        self.worksheet.write(0,2,'N_Features')
        self.worksheet.write(0,3,'Fitness')
        self.worksheet.write(0,4,'Time')
    
    def attributs_to_flip(self,nb_att):
      
        return list(range(nb_att))
    
    def run(self,alpha,beta):
        dim=self.fsd.nb_attribs
        t_init = time.time()
        lb = -10*np.ones([dim, 1]) #下边界
        ub = 10*np.ones([dim, 1])#上边界
        for itr in range(1,self.nb_exec+1):
            print ("Execution N:{0}".format(str(itr)))
            self.fsd = FsProblem(self.df)
            t1 = time.time()
            test=Test(self.fsd,alpha,beta)
            best=test.BES(self.dragonfly_num,lb,ub,self.max_iter)
            
            t2 = time.time()
            print("Time elapsed for execution N:{0} : {1:.2f} s\n".format(itr,t2-t1))
            self.worksheet.write(itr, 0, itr)
            self.worksheet.write(itr, 1, best[0])
            self.worksheet.write(itr, 2, best[1])
            self.worksheet.write(itr, 3, best[2])
            self.worksheet.write(itr, 4, t2-t1)
            
            num = 0
            column = 0
            for item in best[3]:
                if num%10==0:
                    self.worksheet2.write(column, (itr-1), item)
                    column = column +1
                num = num+1
          
        t_end = time.time()
        print ("Total execution time for dataset {0} is {1:.2f} s".format(self.dataset_name,t_end-t_init))
        self.workbook.close()
        
# Dataset
datasets = ['Exactly-II','Exactly','HeartEW','M-of-N','Breastcancer','BreastEW','Congress','Heart-StatLog','Hepatitis','Hillvalley','Lung','Lung-Cancer','Lymphography','Movementlibras','Sonar','Vowel','WDBC','Wine','Zoo','Dermatology','Krvskp','semeion','tic-tac-toe']

data_loc_path = "../datasets/"
#执行次数， 蜻蜓的数量， 最大迭代次数
nbr_exec =30
dragonfly_num = 24
max_iter=101
alpha=0.99
beta=0.01
for dataset in datasets:
    location = data_loc_path + dataset + ".csv"
    instance = FSData(location,nbr_exec,dragonfly_num,max_iter)
    instance.run(alpha,beta)

